# Imports

In [ ]:
# The Only imports I will need
import numpy as np
import scipy

# Load cleaned data

## ndarrays

In [ ]:
x_train = np.load("data/train.npy")
x_test = np.load("data/test.npy")

In [ ]:
x_train.shape

## Dictionaries

In [ ]:
f = open("data/id_index.txt","r")
id_index = eval(f.read())
f.close()
f = open("data/index_id.txt","r")
index_id = eval(f.read())
f.close()

# Build the Auto Encoder

In [ ]:
test = x_train[:3]
test

In [ ]:
class Neural_Net:
    def __init__(self):

        # self.lr = 0.000001
        self.lr = 0.1

        # Input layer size
        self.Input = 2540

        # 2 Encoder hidden layers
        self.Dense1 = 1024
        self.Dense2 = 1024

        # Dimentions of the latent space 
        self.Latent = 32

        # 2 Decoder hidden layers
        self.Dense3 = 1024
        self.Dense4 = 1024
        
        # Output same as input
        self.Output = 2540

        # Weights
        np.random.seed(73)
        self.weights = {}

        self.weights["i_d1"] = np.random.randn(self.Input, self.Dense1) 
        self.weights['b_d1'] = np.random.randn(self.Dense1,)

        self.weights["d1_d2"] = np.random.randn(self.Dense1, self.Dense2) 
        self.weights['b_d2'] = np.random.randn(self.Dense2,)

        self.weights["d2_late"] = np.random.randn(self.Dense2, self.Latent) 
        self.weights['b_late'] = np.random.randn(self.Latent,)

        self.weights["late_d3"] = np.random.randn(self.Latent, self.Dense3) 
        self.weights['b_d3'] = np.random.randn(self.Dense3,)

        self.weights["d3_d4"] = np.random.randn(self.Dense3, self.Dense4) 
        self.weights['b_d4'] = np.random.randn(self.Dense4,)

        self.weights["d4_out"] = np.random.randn(self.Dense4, self.Output) 
        self.weights['b_out'] = np.random.randn(self.Output,)
    
    def relu(self,x):
        return np.maximum(0,x)

    def relu_gradient(self, x):
        return np.where(x >= 0, 1, 0)
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))

    def sigmoid_gradient(self,x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def feedforward(self,x):
        # Encoder
        z_d1 = (np.dot(x, self.weights["i_d1"])) + self.weights["b_d1"]
        a_d1 = self.sigmoid(z_d1)

        z_d2 = (np.dot(a_d1, self.weights["d1_d2"])) + self.weights["b_d2"]
        a_d2 = self.sigmoid(z_d2)

        # Latent Space
        z_late = (np.dot(a_d2, self.weights["d2_late"])) + self.weights["b_late"]
        a_late = self.sigmoid(z_late)

        # Decoder
        z_d3 = (np.dot(a_late, self.weights["late_d3"])) + self.weights["b_d3"]
        a_d3 = self.sigmoid(z_d3)

        z_d4 = (np.dot(a_d3, self.weights["d3_d4"])) + self.weights["b_d4"]
        a_d4 = self.sigmoid(z_d4)

        z_out = (np.dot(a_d4, self.weights["d4_out"])) + self.weights["b_out"]
        a_out = self.relu(z_out)
        output = a_out

        self.layer_outputs = {}
        self.layer_outputs["z_d1"] = z_d1
        self.layer_outputs["a_d1"] = a_d1
        self.layer_outputs["z_d2"] = z_d2
        self.layer_outputs["a_d2"] = a_d2
        self.layer_outputs["z_late"] = z_late
        self.layer_outputs["a_late"] = a_late
        self.layer_outputs["z_d3"] = z_d3
        self.layer_outputs["a_d3"] = a_d3
        self.layer_outputs["z_d4"] = z_d4
        self.layer_outputs["a_d4"] = a_d4
        self.layer_outputs["z_out"] = z_out
        self.layer_outputs["a_out"] = a_out
        return output

    def predict(self,x):
        output = self.feedforward(x)
        return output

    def get_loss(self, output, x):
        return np.sum(((x - output)**2) * 0.5)

    def get_loss_gradient(self, output, x):
        return (x - output)

    def back_prop(self, output, x):
        # Take the dx of the loss function
        dx_dloss = self.get_loss_gradient(output, x)

        dx_da_out = self.relu_gradient(output)
        dx_dz_out = dx_dloss * dx_da_out
        dx_dw_d4_out = np.dot(self.layer_outputs['a_d4'].T, dx_dz_out)
        dx_dw_b_out = np.sum(dx_dz_out, axis=0)

        dx_da_d4 = np.dot(dx_dz_out, self.weights["d4_out"].T)
        dx_dz_d4 = dx_da_d4 * self.sigmoid_gradient(self.layer_outputs["z_d4"])
        dx_dw_d3_d4 = self.layer_outputs["z_d3"].T.dot(dx_dz_d4)
        dx_dw_b_d4 = np.sum(dx_dz_d4, axis=0)

        dx_da_d3 = np.dot(dx_dz_d4, self.weights["d3_d4"].T)
        dx_dz_d3 = dx_da_d3 * self.sigmoid_gradient(self.layer_outputs["z_d3"])
        dx_dw_late_d3 = self.layer_outputs["z_late"].T.dot(dx_dz_d3)
        dx_dw_b_d3 = np.sum(dx_dz_d3, axis=0)

        dx_da_late = np.dot(dx_dz_d3, self.weights["late_d3"].T)
        dx_dz_late = dx_da_late * self.sigmoid_gradient(self.layer_outputs["z_late"])
        dx_dw_d2_late = self.layer_outputs["z_d2"].T.dot(dx_dz_late)
        dx_dw_b_late = np.sum(dx_dz_late, axis=0)

        dx_da_d2 = np.dot(dx_dz_late, self.weights["d2_late"].T)
        dx_dz_d2 = dx_da_d2 * self.sigmoid_gradient(self.layer_outputs["z_d2"])
        dx_dw_d1_d2 = self.layer_outputs["z_d1"].T.dot(dx_dz_d2)
        dx_dw_b_d2 = np.sum(dx_dz_d2, axis=0)

        dx_da_d1 = np.dot(dx_dz_d2, self.weights["d1_d2"].T)
        dx_dz_d1 = dx_da_d1 * self.sigmoid_gradient(self.layer_outputs["z_d1"])
        dx_dw_i_d1 = x.T.dot(dx_dz_d1)
        dx_dw_b_d1 = np.sum(dx_dz_d1, axis=0)


        # Update the weights
        self.weights["i_d1"] += self.lr * dx_dw_i_d1
        self.weights['b_d1'] += self.lr * dx_dw_b_d1

        self.weights["d1_d2"] += self.lr * dx_dw_d1_d2
        self.weights['b_d2'] += self.lr * dx_dw_b_d2

        self.weights["d2_late"] += self.lr * dx_dw_d2_late
        self.weights['b_late'] += self.lr * dx_dw_b_late

        self.weights["late_d3"] += self.lr * dx_dw_late_d3
        self.weights['b_d3'] += self.lr * dx_dw_b_d3

        self.weights["d3_d4"] += self.lr * dx_dw_d3_d4
        self.weights['b_d4'] += self.lr * dx_dw_b_d4

        self.weights["d4_out"] += self.lr * dx_dw_d4_out
        self.weights['b_out'] += self.lr * dx_dw_b_out

    def train(self, x, epochs=5, batchsize=1024):
        for epoch in range(epochs):
            total_loss = []
            for batch in range(round(len(x)/batchsize)):
                minibatch = x[batch*batchsize:(batch+1)*batchsize]
                output = self.feedforward(minibatch)
                self.back_prop(output, minibatch)
                loss = self.get_loss(output, minibatch)
                total_loss.append(loss)
                self.lr *= 0.9925
                print(f"Epoch {epoch+1}: Batch {batch+1} out of {round(len(x)/batchsize)}: Loss = {loss:.2f}")
            total_loss = np.array(total_loss)
            print(f"Epoch {epoch+1} of {epochs}: Loss = {np.mean(total_loss):.2f}")

In [ ]:
# def feeddata(x):
#     sigmoid = 1/(1+np.exp(-x))
#     return np.where(sigmoid == 0.5, 0, sigmoid)

In [ ]:
Brain = Neural_Net()

In [ ]:
test

In [ ]:
# feeddata(test)

In [ ]:
pred = Brain.predict(test)
print(pred.shape)
print(pred)

In [ ]:
Brain.train(x_train, epochs=5, batchsize=1024)

In [ ]:
pred = Brain.predict(test)
print(pred.shape)
print(pred)

# print(test[0][50:100])
# print(pred[0][0:1000])
# print(pred[0][1000:2000])
# print(pred[0][2000:])

In [ ]:
# take the top row (1st user) and add an index to their values
pred_index = np.insert(np.atleast_2d(pred[1]), 0, [num for num in range(np.atleast_2d(pred[1]).shape[1])], axis=0).T
pred_index

In [ ]:
# Return the top n scores
top_n = 5
top_n_pred = pred_index[pred_index[:,1].argsort()][-top_n:]
# top_n_pred.reverse()
print(top_n_pred)

In [ ]:
# convert top n scores to list and replace using dictionary keys
top_n_output = [index_id[num] for num in top_n_pred[:,0]]
print(top_n_output)

In [ ]:
# 2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
# 593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
# 356,Forrest Gump (1994),Comedy|Drama|Romance|War
# 296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
# 318,"Shawshank Redemption, The (1994)",Crime|Drama